# Utiliser des modèles

De plus en plus de modèles préentrainés

In [7]:
#!pip install spacy

## Charger les données

In [8]:
import pandas as pd
df = pd.read_csv("../data/dataframe.csv")
df.head()

,Unnamed: 0,numero,date,texte,texte_net
0,0,3063,03/06/2020,"</p><p align=""CENTER""> RÔLE DES COLLECTIVITÉS ...",RÔLE DES COLLECTIVITÉS LOCALES DANS LA LUTTE C...
1,1,3599,09/12/2020,"</p><p align=""CENTER""> CONTRÔLE DES EXPORTATIO...",CONTRÔLE DES EXPORTATIONS D'ARMEMENT M. le pré...
2,2,89,10/08/2017,"</p><p align=""CENTER""> TAXE DE SÉJOUR <a name=...",TAXE DE SÉJOUR M. le président. La parole est ...
3,3,348,29/11/2017,"</p><p align=""CENTER""> LISTE NOIRE DES PARADIS...",LISTE NOIRE DES PARADIS FISCAUX M. le présiden...
4,4,3433,21/10/2020,"</p><p align=""CENTER""> LUTTE CONTRE LE FINANCE...",LUTTE CONTRE LE FINANCEMENT DU TERRORISME M. l...


## Une approche intégrée avec SpaCy

- `SpaCy` a des modèles entraînés pour les NER
- Par exemple pour le français, [plusieurs modèles sont disponibles](https://spacy.io/models/fr)
    - Avec des architectures différentes
- Une bibliothèque qui donne un framework commun.


Une étape : télécharger des modèles

https://github.com/explosion/spacy-models/releases/tag/fr_core_news_md-3.8.0

In [9]:
# !python -m spacy download fr_core_news_md

Les utiliser

In [10]:
#df.loc[0, "texte"]

In [11]:
import spacy
nlp = spacy.load("fr_core_news_md")

In [12]:
doc = nlp(df.loc[0, "texte_net"])

In [13]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

### Manipuler les représentations du texte

In [14]:
for token in doc[0:20]:
    print(f"{token.text:<15} | lemma: {token.lemma_} | POS: {token.pos_}")

RÔLE            | lemma: rôle | POS: NOUN
DES             | lemma: de | POS: ADP
COLLECTIVITÉS   | lemma: collectivité | POS: NOUN
LOCALES         | lemma: local | POS: ADJ
DANS            | lemma: dans | POS: ADP
LA              | lemma: le | POS: DET
LUTTE           | lemma: lutte | POS: NOUN
CONTRE          | lemma: contre | POS: ADP
LA              | lemma: le | POS: DET
CRISE           | lemma: crise | POS: NOUN
M.              | lemma: m. | POS: NOUN
                | lemma:   | POS: SPACE
le              | lemma: le | POS: DET
président       | lemma: président | POS: NOUN
.               | lemma: . | POS: PUNCT
La              | lemma: le | POS: DET
parole          | lemma: parole | POS: NOUN
est             | lemma: être | POS: AUX
à               | lemma: à | POS: ADP
M.              | lemma: m. | POS: NOUN


In [15]:
doc[100].pos_

'NOUN'

In [16]:
for ent in doc.ents[0:10]:
    print(f"{ent.text:<25} | label: {ent.label_}")

DES COLLECTIVITÉS LOCALES | label: MISC
LUTTE CONTRE LA CRISE     | label: MISC
M.                        | label: PER
Patrice Verchère          | label: PER
Patrice Verchère          | label: PER
Cours                     | label: MISC
M. Darmanin               | label: PER
Applaudissements          | label: ORG
LR.)M.                    | label: ORG
Pierre Cordier            | label: PER


In [17]:
displacy.render(doc[0:100], style="dep", jupyter=True, options={"compact": True})


### Un usage : récupérer uniquement les verbes

In [18]:
def get_verbs(doc):
    """
    Get the verbs from a spacy doc
    """
    return [token.lemma_ for token in doc if token.pos_ == "VERB"]

tmp =  df["texte_net"][0:100].apply(lambda x: get_verbs(nlp(x)))

In [19]:
from collections import Counter

Counter([j for i in tmp for j in i if j]).most_common(20)

[('faire', 302),
 ('avoir', 291),
 ('pouvoir', 238),
 ('devoir', 228),
 ('être', 154),
 ('permettre', 115),
 ('savoir', 113),
 ('dire', 112),
 ('aller', 107),
 ('vouloir', 100),
 ('falloir', 100),
 ('prendre', 85),
 ('agir', 82),
 ('mettre', 77),
 ('rappeler', 64),
 ('passer', 64),
 ('charger', 61),
 ('venir', 61),
 ('travailler', 59),
 ('–', 58)]

### Un autre usage : nettoyer des textes

Plutôt que faire de la tokenisation brutale, nettoyer en prenant uniquement les lemmes puis faire un TF-IDF

In [20]:
# exercice

## Aller utiliser d'autres modèles sur huggingface

Commençons par faire un tour sur Huggingface

### Prédire des entités nommées

Utilsons le modèle [GliNER disponible sur HuggingFace](https://github.com/urchade/GLiNER)

Ou sa version plus récente [GliNer](https://huggingface.co/knowledgator/gliner-multitask-large-v0.5)

In [21]:
#!pip install gliner

In [23]:
from gliner import GLiNER

# on récupère le modèle
model = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")


ValueError: Unknown span mode token_level

In [ ]:

text = df.loc[2, "texte_net"]

labels = ["politicien ou politicienne"]

entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

/opt/anaconda3/lib/python3.11/site-packages/gliner/data_processing/processor.py:410: UserWarning: Sentence of length 931 has been truncated to 768
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Benjamin Dirx => politicien ou politicienne
Benjamin Dirx => politicien ou politicienne
Benjamin Dirx => politicien ou politicienne
Bruno Le Maire => politicien ou politicienne
Benjamin Dirx => politicien ou politicienne
Bruno Le Maire => politicien ou politicienne
Thierry Benoit => politicien ou politicienne
Jean-Luc Mélenchon => politicien ou politicienne
Bruno Le Maire => politicien ou politicienne
Gérald Darmanin => politicien ou politicienne
Claude Goasguen => politicien ou politicienne
Bruno Le Maire => politicien ou politicienne
Jean-Paul Lecoq => politicien ou politicienne
Bruno Le Maire => politicien ou politicienne


### De nombreux modèles et la possibilité d'en entraîner

PAr exemple : https://huggingface.co/NousResearch/Minos-v1

### Analyse de sentiment

Une question : **quelles sont les prises de paroles les plus négatives ?**

- Embarras du choix
    - Par ex : [🚀 distilbert-based Multilingual Sentiment Classification Model
](https://huggingface.co/tabularisai/multilingual-sentiment-analysis)
- Comprendre le modèle / ce qu'il fait
- Importance d'évaluer son résultat

## Utiliser des modèles extérieurs

Il faut un endpoint :

- Ollama
- OpenAI
- ...

In [ ]:
import requests

# définir l'endpoint
endpoint = ""

# Définir la prompt de classification
prompt = """Est-ce que ce texte est positif ou négatif : Je ne comprends rien au NLP. 
Ne réponds que POSITIF ou NEGATIF."""

# Envoi de la requête à Ollama
response = requests.post(
    endpoint,
    json={
        'model': 'llama3.3',
        'prompt': prompt,
        'stream': False  # stream=False pour avoir une réponse simple
    }
)

# Traitement de la réponse
result = response.json()
result

{'model': 'llama3.3',
 'created_at': '2025-05-20T15:57:20.053721724Z',
 'response': 'NEGATIF',
 'done': True,
 'done_reason': 'stop',
 'context': [128006,
  882,
  128007,
  271,
  14101,
  54312,
  1744,
  3846,
  69067,
  1826,
  20940,
  333,
  6033,
  308,
  19395,
  50848,
  551,
  14465,
  841,
  60946,
  82,
  55455,
  8065,
  452,
  12852,
  13,
  720,
  8989,
  75871,
  82,
  1744,
  27592,
  964,
  2843,
  6033,
  85165,
  835,
  2843,
  13,
  128009,
  128006,
  78191,
  128007,
  271,
  98227,
  835,
  2843],
 'total_duration': 178492331,
 'load_duration': 14576150,
 'prompt_eval_count': 45,
 'prompt_eval_duration': 44405857,
 'eval_count': 4,
 'eval_duration': 118533952}